# 清理

本笔记本演示如何清理在之前的笔记本中集中创建的所有资源。

In [ ]:
import boto3
import json

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [ ]:
%store -r

请注意，这些步骤的决定条件在于，它们是否在其他步骤开始之前全部完成。如果收到错误，请等待一两分钟，然后重试。

## 清理活动

从清理活动开始。您需要先删除活动，然后才能删除活动所基于的解决方案版本。

以下代码将删除您已创建的活动。

In [ ]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

## 清理事件跟踪器

接下来，清理所有事件跟踪器。以下代码将删除您的事件跟踪器。

In [ ]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

## 清理筛选器

接下来，清理所有筛选器。以下代码将对您创建的所有筛选器进行迭代并删除。请注意，如果您收到活动相关错误，则可以等待 2 分钟，然后再次尝试以下单元格。

注意：在活动删除完成之前，您将无法删除筛选器，这可能需要几分钟时间

In [ ]:
for filter_arn in meta_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in decade_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

## 清理解决方案

接下来，清理解决方案。以下代码将删除已创建的解决方案。

In [ ]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

## 清理数据集

接下来，清理数据集。以下代码将删除项目和交互数据集。

In [ ]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

## 清理架构

接下来，清理架构。创建交互和项目模式，然后删除。

In [ ]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

## 清理数据集组

最后，清理数据集组：

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

## 清除 S3 存储桶和 IAM 角色

首先删除角色，然后清空存储桶，再删除存储桶。

In [ ]:
iam = boto3.client('iam')

确定要删除的角色的名称。

不能删除仍附加有策略的 IAM 角色。因此，在您确定了相关角色之后，让我们列出该角色的附加策略。

In [ ]:
iam.list_attached_role_policies(
    RoleName = role_name
)

您需要使用下面的代码分离上面结果中的策略。对附加的每个策略重复此操作。

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

In [ ]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

最后，您应该能够删除 IAM 角色。

In [ ]:
iam.delete_role(
    RoleName = role_name
)

要删除 S3 存储桶，首先它需要是空的。删除 S3 存储桶最简单的方法是在 AWS 控制台中导航到 S3，删除存储桶中的对象，然后删除 S3 存储桶本身。